In [11]:
library(Signac)
library(Seurat)
library(EnsDb.Hsapiens.v86)
library(SeuratDisk)
library(dplyr)
library(BSgenome.Hsapiens.UCSC.hg38)

In [ ]:
ifnb <- readRDS("pd_processed_rna+atac.rds")

In [ ]:
ifnb <- subset(ifnb, celltype == "microglia")

In [14]:
seqlevels(Annotation(ifnb))

[1] "chrX"  "chr20" "chr1"  "chr6"  "chr3"  "chr7"  "chr12" "chr11" "chr4" 
[10] "chr17" "chr2"  "chr16" "chr8"  "chr19" "chr9"  "chr13" "chr14" "chr5" 
[19] "chr22" "chr10" "chrY"  "chr18" "chr15" "chr21" "chrMT"

In [15]:
DefaultAssay(ifnb) <- "ATAC"
# first compute the GC content for each peak
ifnb <- RegionStats(ifnb, genome = BSgenome.Hsapiens.UCSC.hg38)

Warning message in RegionStats.default(object = regions, genome = genome, verbose = verbose, :
"Not all seqlevels present in supplied genome"


In [ ]:
idents.plot <- c('microglia')
Idents(ifnb) <- idents.plot

### Helpers

In [7]:
library(ggplot2)
save_link_counts <- function(ifnb, gene_list, file_prefix){
    # link peaks to genes
    ifnb <- LinkPeaks(
      object = ifnb,
      peak.assay = "ATAC",
      expression.assay = "SCT",
      genes.use = gene_list
    )

    peak_gene_links <- Links(ifnb[["ATAC"]])
    df <- as.data.frame(peak_gene_links)

    gene_link_counts <- df %>%
      filter(pvalue < 0.05) %>%
      group_by(gene) %>%
      summarise(link_count = n())

    write.csv(gene_link_counts, paste0(file_prefix, "gene_link_counts.csv"), row.names = FALSE)
    
    # Return both the original df and gene_link_counts as a list
    return(gene_link_counts)
}

Warning message:
"package 'ggplot2' was built under R version 4.3.3"


In [8]:
library(ggplot2)
plot_coverage <- function(ifnb, gene_list, file_prefix){
    DefaultAssay(ifnb) <- "ATAC"
    # link peaks to genes
    ifnb <- LinkPeaks(
      object = ifnb,
      peak.assay = "ATAC",
      expression.assay = "SCT",
      genes.use = gene_list
    )

    library(ggplot2)
    for (i in gene_list) { 
        print(i)
        
        # Try running CoveragePlot(), skip if error occurs
        tryCatch({
            p1 <- CoveragePlot(
                object = ifnb,
                region = i,
                features = i,
                expression.assay = "SCT",
                links = TRUE,
                link.assay = "ATAC",
                idents = idents.plot,
                extend.upstream = 50000,
                extend.downstream = 500000
            )
    
            # Save the plot
            ggsave(paste0(file_prefix, i, ".pdf"), p1, width = 15)
        }, error = function(e) {
            message(paste("Skipping:", i, "- Error encountered:", e$message))
        })
    }

}

### Marker Genes; MG0

In [ ]:
# degs RBD vs HC
library(readxl)
library(dplyr)

# Define file path
file_path <- "../../Annotate_Mic_Subtype/11_clusters/Marker_Genes.xlsx"

# Read the specific sheet "Cluster_0"
gene_data <- read_excel(file_path, sheet = "Cluster_0")

# Ensure column names are correctly recognized
colnames(gene_data) <- c("gene", "p_val", "avg_log2FC", "pct.1", "pct.2", "p_val_adj")

# Filter genes based on criteria: p_val_adj < 0.001 and avg_log2FC > 0.25
filtered_genes <- gene_data %>%
  filter(p_val_adj < 0.001, avg_log2FC > 0.25)

# Select the top 50 genes with the smallest p_val_adj (keep original order)
top_genes <- head(filtered_genes$gene, 50)

top_genes

New names:
• `` -> `...1`


[1] "PLXDC2"   "SRGAP2"   "CHST11"   "MEF2A"    "LRMDA"    "EPB41L2" 
 [7] "ITPR2"    "DIAPH2"   "ST6GAL1"  "FOXN3"    "DOCK8"    "ARHGAP26"
[13] "SLC1A3"   "MGAT4A"   "APBB1IP"  "ARHGAP24" "SKAP2"    "FYB1"    
[19] "NCK2"     "SH3RF3"   "FAM49B"   "RNASET2"  "BMP2K"    "RUNX1"   
[25] "DISC1"    "LHFPL2"   "SLC9A9"   "TBXAS1"   "DOCK2"    "DENND3"  
[31] "ATP8B4"   "FGD4"     "PTPRC"    "ETV6"     "GNAQ"     "KCNQ3"   
[37] "SORL1"    "MED13L"   "IPCEF1"   "LDLRAD4"  "ARHGAP15" "SFMBT2"  
[43] "MAML2"    "LNCAROD"  "SSH2"     "FMN1"     "JAZF1"    "FKBP5"   
[49] "TAB2"     "MAML3"

In [ ]:
file_prefix <- "../../Signac_plots/PD/Marker_Genes_Cluster0/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

save_link_counts(ifnb, top_genes, file_prefix)

Directory already exists: ../Signac_plots/PD/Marker_Genes_Cluster0/

Testing 50 genes and 17950 peaks

Found gene coordinates for 47 genes

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


gene,link_count
<chr>,<int>
DENND3,1
DOCK8,1
EPB41L2,1
FAM49B,1
FGD4,1
FKBP5,1
FOXN3,2
ITPR2,1
LDLRAD4,1


In [ ]:
file_prefix <- "../../Signac_plots/PD/Marker_Genes_Cluster0/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

plot_coverage(ifnb, top_genes, file_prefix)

Directory already exists: ../Signac_plots/PD/Marker_Genes_Cluster0/

Testing 50 genes and 17950 peaks

Found gene coordinates for 47 genes

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


[1] "PLXDC2"


Saving 15 x 6.67 in image


[1] "SRGAP2"


Saving 15 x 6.67 in image
Warning message:
"Removed 44 rows containing missing values or values outside the scale range
(`geom_segment()`)."


...

Saving 15 x 6.67 in image
Warning message:
"Removed 4 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "TAB2"


Saving 15 x 6.67 in image
Warning message:
"Removed 9 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "MAML3"


Saving 15 x 6.67 in image
Warning message:
"Removed 32 rows containing missing values or values outside the scale range
(`geom_segment()`)."


### Marker Genes; MG2

In [ ]:
# degs RBD vs HC
library(readxl)
library(dplyr)

# Define file path
file_path <- "../../Annotate_Mic_Subtype/11_clusters/Marker_Genes.xlsx"

# Read the specific sheet "Cluster_0"
gene_data <- read_excel(file_path, sheet = "Cluster_2")

# Ensure column names are correctly recognized
colnames(gene_data) <- c("gene", "p_val", "avg_log2FC", "pct.1", "pct.2", "p_val_adj")

# Filter genes based on criteria: p_val_adj < 0.001 and avg_log2FC > 0.25
filtered_genes <- gene_data %>%
  filter(p_val_adj < 0.001, avg_log2FC > 0.25)

# Select the top 50 genes with the smallest p_val_adj (keep original order)
top_genes <- head(filtered_genes$gene, 50)

top_genes

New names:
• `` -> `...1`


[1] "FKBP5"    "SSH2"     "NEAT1"    "CD163"    "FMN1"     "CELF2"   
 [7] "ARHGAP24" "MKI67"    "AKAP13"   "SLC1A3"   "MELK"     "FGD4"    
[13] "BUB1"     "TOP2A"    "ETV6"     "SMAP2"    "NEIL3"    "F13A1"   
[19] "CDK1"     "ACSL1"    "BUB1B"    "KNL1"     "SRGN"     "ASPM"    
[25] "TAB2"     "NCAPG"    "KIF4A"    "CD163L1"  "KIF14"    "CTSB"    
[31] "FAM111B"  "HJURP"    "KIF11"    "SLC11A1"  "CEP55"    "HIF1A"   
[37] "CENPF"    "TPX2"     "RGS1"     "NDC80"    "SBF2"     "IFRD1"   
[43] "KCNMA1"   "ABR"      "DIAPH3"   "INSIG1"   "KIF23"    "TTK"     
[49] "RBM47"    "KIF18B"

In [ ]:
file_prefix <- "../../Signac_plots/PD/Marker_Genes_Cluster2/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

save_link_counts(ifnb, top_genes, file_prefix)

Directory already exists: ../Signac_plots/PD/Marker_Genes_Cluster2/

Testing 24 genes and 17950 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


gene,link_count
<chr>,<int>
ABR,2
ACSL1,1
FGD4,1
FKBP5,1
IFRD1,2
INSIG1,1
KCNMA1,1
NEAT1,4
RBM47,1


In [ ]:
file_prefix <- "../../Signac_plots/PD/Marker_Genes_Cluster2/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

plot_coverage(ifnb, top_genes, file_prefix)

Directory already exists: ../Signac_plots/PD/Marker_Genes_Cluster2/

Testing 24 genes and 17950 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


[1] "FKBP5"


Saving 15 x 6.67 in image
Warning message:
"Removed 4 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "SSH2"


Saving 15 x 6.67 in image
Warning message:
"Removed 170 rows containing missing values or values outside the scale range
(`geom_segment()`)."


...

Saving 15 x 6.67 in image


[1] "RBM47"


Saving 15 x 6.67 in image


[1] "KIF18B"


Saving 15 x 6.67 in image


### Marker Genes; MG3

In [ ]:
# degs RBD vs HC
library(readxl)
library(dplyr)

# Define file path
file_path <- "../../Annotate_Mic_Subtype/11_clusters/Marker_Genes.xlsx"

# Read the specific sheet "Cluster_0"
gene_data <- read_excel(file_path, sheet = "Cluster_3")

# Ensure column names are correctly recognized
colnames(gene_data) <- c("gene", "p_val", "avg_log2FC", "pct.1", "pct.2", "p_val_adj")

# Filter genes based on criteria: p_val_adj < 0.001 and avg_log2FC > 0.25
filtered_genes <- gene_data %>%
  filter(p_val_adj < 0.001, avg_log2FC > 0.25)

# Select the top 50 genes with the smallest p_val_adj (keep original order)
top_genes <- head(filtered_genes$gene, 50)

top_genes

New names:
• `` -> `...1`


[1] "IL1RAPL1"    "ST18"        "PTPRD"       "SLC24A2"     "SPOCK1"     
 [6] "PPP2R2B"     "FRMD5"       "SPOCK3"      "TTLL7"       "MAP7"       
[11] "PEX5L"       "RNF220"      "KCNH8"       "EDIL3"       "PDE1C"      
[16] "CTNNA3"      "PRUNE2"      "ZNF536"      "DNAJC6"      "C10orf90"   
[21] "SH3GL3"      "PALM2-AKAP2" "CLMN"        "SLC7A14-AS1" "MOBP"       
[26] "ENPP2"       "TF"          "SLCO1A2"     "LINC01608"   "NCAM1"      
[31] "UGT8"        "NCAM2"       "SHROOM4"     "ANK3"        "TMTC2"      
[36] "MAGI2"       "TMEFF2"      "ANO4"        "LIMCH1"      "UNC5C"      
[41] "PTPRK"       "CREB5"       "COBL"        "NFASC"       "KLHL32"     
[46] "COL4A5"      "CLDN11"      "ABCA8"       "AC008571.2"  "HHIP"

In [ ]:
file_prefix <- "../../Signac_plots/PD/Marker_Genes_Cluster3/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

save_link_counts(ifnb, top_genes, file_prefix)

Directory already exists: ../Signac_plots/PD/Marker_Genes_Cluster3/

Testing 49 genes and 17950 peaks

Found gene coordinates for 48 genes

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


gene,link_count
<chr>,<int>
CREB5,1
FRMD5,1
LIMCH1,1
MAP7,1
MOBP,1
PEX5L,1
RNF220,4
SH3GL3,1
SLCO1A2,1


In [ ]:
file_prefix <- "../../Signac_plots/PD/Marker_Genes_Cluster3/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

plot_coverage(ifnb, top_genes, file_prefix)

Directory already exists: ../Signac_plots/PD/Marker_Genes_Cluster3/

Testing 49 genes and 17950 peaks

Found gene coordinates for 48 genes

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


[1] "IL1RAPL1"


Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Saving 15 x 6.67 in image
Warning message:
"Removed 2 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "ST18"


Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Saving 15 x 6.67 in image


...

Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Saving 15 x 6.67 in image
Warning message:
"Removed 69 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "AC008571.2"


Skipping: AC008571.2 - Error encountered: Gene not found



[1] "HHIP"


Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Saving 15 x 6.67 in image


### DEGs PD vs HC; MG0

In [ ]:
# update plot_coverage function
library(Signac)
library(patchwork)   # lets us manipulate the assembled plot
library(ggplot2)

plot_coverage <- function(ifnb, gene_list, file_prefix,
                          idents.plot = NULL,
                          extend.up = 5e4, extend.down = 5e5) {

  DefaultAssay(ifnb) <- "ATAC"
  ifnb <- LinkPeaks(ifnb,
                    peak.assay       = "ATAC",
                    expression.assay = "SCT",
                    genes.use        = gene_list)

  for (g in gene_list) {
    message("Plotting ", g)

    tryCatch({

      cov <- CoveragePlot(
             ifnb,
             region            = g,
             features          = g,
             expression.assay  = "SCT",
             links             = FALSE,
             link.assay        = "ATAC",
             idents            = idents.plot,
             extend.upstream   = extend.up,
             extend.downstream = extend.down
           )
       arc  <- LinkPlot(ifnb, region = g, assay = "ATAC",
                  extend.upstream = 5e4, extend.downstream = 5e5) +
       scale_colour_gradient2(low = "blue", mid = "grey",
                               high = "#ca0020", midpoint = 0,
                               limits = c(-0.05, 0.1), name = "Link\nscore") +
        guides(colour = guide_colourbar(title.position = "top")) +
        theme(legend.position = "right")

       final <- cov / arc

       ggsave(paste0(file_prefix, g, ".pdf"), final, width = 15, height = 6)

    }, error = function(e) {
      message("Skipping ", g, ": ", e$message)
    })
  }
}

Warning message:
"package 'patchwork' was built under R version 4.3.3"
Warning message:
"package 'ggplot2' was built under R version 4.3.3"


In [ ]:
# degs RBD vs HC
library(readxl)
library(dplyr)

# Define file path
file_path <- "../../DEG_GO_Analysis/DEGs_PD_vs_HC/Significant_DEGs_PD_vs_HC.xlsx"

# Read the specific sheet "Cluster_0"
gene_data <- read_excel(file_path, sheet = "Cluster_0")

# Ensure column names are correctly recognized
colnames(gene_data) <- c("gene", "p_val", "avg_log2FC", "pct.1", "pct.2", "p_val_adj")

# Filter genes based on criteria: p_val_adj < 0.001 and avg_log2FC > 0.25
filtered_genes <- gene_data %>%
  filter(p_val_adj < 0.001, avg_log2FC > 0.25)

# Select the top 50 genes with the smallest p_val_adj (keep original order)
top_genes <- head(filtered_genes$gene, 50)

top_genes

New names:
• `` -> `...1`


[1] "F13A1"      "UTY"        "LINC00278"  "NLGN4Y"     "KCNQ3"     
 [6] "USP9Y"      "GLDN"       "ESRRG"      "DLEU1"      "TTC7A"     
[11] "IFI44L"     "PRKY"       "DLEU7"      "ADGRB3"     "LINC01684" 
[16] "APOC1"      "TMEM163"    "TTTY14"     "NAV3"       "AC009432.2"
[21] "DDX60L"     "P2RY14"     "MX1"        "MTUS1"      "MGAT4A"    
[26] "AC066613.2" "HDAC9"      "EPHA1-AS1"  "CARD11"     "HECTD2"    
[31] "TANC1"      "EPSTI1"     "MYCBP2"     "MIR31HG"    "SIGLEC1"   
[36] "LIPG"       "ANKRD36"    "IFI44"      "CXADR"

In [ ]:
# read DEGs from RBD vs HC, which is already a union of the degs from RBD and PD vs HC
deg = read.csv("../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster0/gene_link_counts.csv")$gene

In [ ]:
file_prefix <- "../../Signac_plots/PD/DEGs_PD_vs_HC_Cluster0/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

df = save_link_counts(ifnb, deg, file_prefix)

Directory already exists: ../Signac_plots/PD/DEGs_PD_vs_HC_Cluster0/

Testing 46 genes and 17950 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


In [ ]:
file_prefix <- "../../Signac_plots/PD/DEGs_PD_vs_HC_Cluster0/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

plot_coverage(ifnb, deg, file_prefix)

Directory already exists: ../Signac_plots/PD/DEGs_PD_vs_HC_Cluster0/

Testing 46 genes and 17950 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


[1] "ACER3"


Saving 15 x 6.67 in image
Warning message:
"Removed 53 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "ACSL1"


Saving 15 x 6.67 in image
Warning message:
"Removed 69 rows containing missing values or values outside the scale range
(`geom_segment()`)."


...

Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Saving 15 x 6.67 in image
Warning message:
"Removed 17 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "USP9Y"


Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Saving 15 x 6.67 in image
Warning message:
"Removed 141 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "ZFY"


Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Saving 15 x 6.67 in image
Warning message:
"Removed 18 rows containing missing values or values outside the scale range
(`geom_segment()`)."


In [ ]:
file_prefix <- "../../Signac_plots/PD/DEGs_PD_vs_HC_Cluster0/"
plot_coverage(ifnb, "GRB2", file_prefix)

Testing 1 genes and 17950 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."
Plotting GRB2

Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Warning message:
"Removed 61 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


In [10]:
plot_coverage(ifnb, "TMEM163", file_prefix)

Testing 1 genes and 17950 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."
No significant links found

Plotting TMEM163

Warning message:
"Removed 125 rows containing missing values or values outside the scale range
(`geom_segment()`)."


### DEGs PD vs HC; MG2

In [ ]:
# Define file path
file_path <- "../../DEG_GO_Analysis/DEGs_PD_vs_HC/Significant_DEGs_PD_vs_HC.xlsx"

# Read the specific sheet "Cluster_0"
gene_data <- read_excel(file_path, sheet = "Cluster_2")

# Ensure column names are correctly recognized
colnames(gene_data) <- c("gene", "p_val", "avg_log2FC", "pct.1", "pct.2", "p_val_adj")

# Filter genes based on criteria: p_val_adj < 0.001 and avg_log2FC > 0.25
filtered_genes <- gene_data %>%
  filter(p_val_adj < 0.001, avg_log2FC > 0.25)

# Select the top 50 genes with the smallest p_val_adj (keep original order)
top_genes <- head(filtered_genes$gene, 50)

top_genes

New names:
• `` -> `...1`


[1] "F13A1"      "DIAPH3"     "IFI44L"     "DLEU7"      "TMEM163"   
 [6] "POLQ"       "NCAPG"      "KNL1"       "LINC00278"  "BUB1B"     
[11] "NAV2"       "CENPI"      "MKI67"      "DLEU1"      "KCNQ3"     
[16] "CIT"        "LRMDA"      "MELK"       "KIF15"      "ANLN"      
[21] "TOP2A"      "UBTD2"      "PRKCA"      "BUB1"       "PARP14"    
[26] "AL078602.1" "NEIL3"      "CXADR"      "HAMP"       "IGSF21"    
[31] "HERC5"      "KIF11"      "ANKRD36"    "PARPBP"     "MS4A4E"    
[36] "MX1"        "STIL"       "NLGN4Y"     "SHCBP1"     "XAF1"      
[41] "KIF4A"      "LINC02057"  "CDK1"       "TTK"        "LINC01684" 
[46] "SPC25"      "CENPK"      "ASPM"       "KCNIP4"     "CCDC144NL"

In [ ]:
# read DEGs from RBD vs HC, which is already a union of the degs from RBD and PD vs HC
deg = read.csv("../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster2/gene_link_counts.csv")$gene

In [ ]:
file_prefix <- "../../Signac_plots/PD/DEGs_PD_vs_HC_Cluster2/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

save_link_counts(ifnb, deg, file_prefix)

Directory already exists: ../Signac_plots/PD/DEGs_PD_vs_HC_Cluster2/

Testing 44 genes and 17950 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


gene,link_count
<chr>,<int>
ADGRE2,1
CIT,3
DENND3,1
DLEU1,2
DLEU7,1
FCGBP,1
FMNL2,3
GAS7,1
HERC5,1


In [ ]:
file_prefix <- "../../Signac_plots/PD/DEGs_PD_vs_HC_Cluster2/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

plot_coverage(ifnb, deg, file_prefix)

Directory already exists: ../Signac_plots/PD/DEGs_PD_vs_HC_Cluster2/

Testing 44 genes and 17950 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


[1] "ACER3"


Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Saving 15 x 6.67 in image
Warning message:
"Removed 53 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "ADGRE2"


Saving 15 x 6.67 in image
Warning message:
"Removed 67 rows containing missing values or values outside the scale range
(`geom_segment()`)."


...

Saving 15 x 6.67 in image
Warning message:
"Removed 63 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "USP9Y"


Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Saving 15 x 6.67 in image
Warning message:
"Removed 141 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "XAF1"


Saving 15 x 6.67 in image
Warning message:
"Removed 55 rows containing missing values or values outside the scale range
(`geom_segment()`)."


### DEGs PD vs HC; MG3

In [ ]:
# Define file path
file_path <- "../../DEG_GO_Analysis/DEGs_PD_vs_HC/Significant_DEGs_PD_vs_HC.xlsx"

# Read the specific sheet "Cluster_0"
gene_data <- read_excel(file_path, sheet = "Cluster_3")

# Ensure column names are correctly recognized
colnames(gene_data) <- c("gene", "p_val", "avg_log2FC", "pct.1", "pct.2", "p_val_adj")

# Filter genes based on criteria: p_val_adj < 0.001 and avg_log2FC > 0.25
filtered_genes <- gene_data %>%
  filter(p_val_adj < 0.001, avg_log2FC > 0.25)

# Select the top 50 genes with the smallest p_val_adj (keep original order)
top_genes <- head(filtered_genes$gene, 50)

top_genes

New names:
• `` -> `...1`


[1] "AC009063.2" "CLECL1"     "TRIM26"     "AC064829.1" "HSPA4L"    
[6] "TOR1AIP2"   "AC104461.1" "EFCAB2"

In [ ]:
# read DEGs from RBD vs HC, which is already a union of the degs from RBD and PD vs HC
deg = read.csv("../../Signac_plots/RBD/DEGs_RBD_vs_HC_Cluster3/gene_link_counts.csv")$gene

In [ ]:
file_prefix <- "../../Signac_plots/PD/DEGs_PD_vs_HC_Cluster3/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

save_link_counts(ifnb, deg, file_prefix)

Directory already exists: ../Signac_plots/PD/DEGs_PD_vs_HC_Cluster3/

Testing 5 genes and 17950 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


gene,link_count
<chr>,<int>
CLECL1,1
GPM6A,1
PLCG2,1
PTGDS,5
TRIM26,3


In [ ]:
file_prefix <- "../../Signac_plots/PD/DEGs_PD_vs_HC_Cluster3/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

plot_coverage(ifnb, deg, file_prefix)

Directory already exists: ../Signac_plots/PD/DEGs_PD_vs_HC_Cluster3/

Testing 5 genes and 17950 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


[1] "ADAMTS18"


Saving 15 x 6.67 in image
Warning message:
"Removed 36 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "CLECL1"


Saving 15 x 6.67 in image
Warning message:
"Removed 57 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


...

Saving 15 x 6.67 in image
Warning message:
"Removed 56 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "PTGDS"


Saving 15 x 6.67 in image
Warning message:
"Removed 59 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "TRIM26"


Saving 15 x 6.67 in image
Warning message:
"Removed 19 rows containing missing values or values outside the scale range
(`geom_segment()`)."


### DAM

In [9]:
dam_ad_genes <- c("CD83", "NAMPT", "HIF1A", "SRGN", "FTH1", "FTL", "RPL32", "TPT1", "RPLP1", 
               "RPS4X", "RPS24", "RPS11", "RPS15", "PLEK", "CCL3", "IL1B", "IFIT2", "B2M", 
               "CD74", "MAMDC2", "EEF1B2", "PDPN", "RPL10A", "RPL34", "RPL35A", "RPL37", 
               "RPL7", "RPL38", "RPS16", "RPL6", "RPS14", "RPS9", "RPL30", "APOE", "RPS25", 
               "RPS20", "EEF1A1", "RPS8", "RPL13", "RPL14", "RPL8", "RPL35", "RPL23", "FAU", 
               "RPS19", "RPL27A", "RPS3", "RPL37A", "ELOVL5", "SOCS6", "CADM1", 
               "ITGAX", "NAP1L1", "CEBPA", "TREM2", "PLXDC2", "SOAT1", "LIPA", "CD84", "CREG1", 
               "FAM20C", "SLC16A3", "ARHGEF7", "GPI", "CUX1", "DHRS3", "CSF2RA", "PSAP", 
               "RASSF3", "RHEB", "BCL2", "NFE2L2", "TLR2", "CHPT1", "FGL2", "EFR3A", "ARAP2", 
               "CLEC7A", "CYBA", "ITM2B", "CD37", "ITGB2", "ARPC3", "HLA-E", "EEF2", "PABPC1", 
               "NPC2", "FCER1G", "RPSA", "TYROBP", "RPL31", "EEF1D", "RPL12", "CHST11", "FCGR3A", 
               "AZIN1", "MFSD1", "GNAS", "NOP58", "LAT2", "CTSZ", "SNX3", "ADAR", "AXL", "BST2", 
               "OXCT1", "SREBF2", "NRP1", "USP12", "GPNMB", "MYO1E", "SPP1", "ELL2", "PADI2", 
               "ATP1B3", "RNF149", "SLC11A1", "KLHL6", "PGK1", "CTSB", "SDCBP")

In [ ]:
file_prefix <- "../../Signac_plots/PD/DAM_AD/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

res <- save_link_counts(ifnb, dam_ad_genes, file_prefix)

Directory already exists: ../Signac_plots/PD/DAM_AD/

Testing 108 genes and 17950 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


In [ ]:
file_prefix <- "../../Signac_plots/PD/DAM_AD/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

plot_coverage(ifnb, dam_ad_genes, file_prefix)

Directory already exists: ../Signac_plots/PD/DAM_AD/

Testing 108 genes and 17950 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, KI270721.1, KI270726.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


[1] "CD83"


Saving 15 x 6.67 in image


[1] "NAMPT"


Saving 15 x 6.67 in image
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


...

Saving 15 x 6.67 in image
Warning message:
"Removed 15 rows containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "LIPA"


Saving 15 x 6.67 in image
Warning message:
"Removed 1 row containing missing values or values outside the scale range
(`geom_segment()`)."


[1] "CD84"


Saving 15 x 6.67 in image
